In [7]:
import requests
import pandas as pd
import os
import json

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from pyspark.sql.functions import current_date, current_timestamp

from pyspark.sql import SparkSession

In [8]:
os.getcwd()

'/home/jovyan/work'

In [9]:
input_url = "https://api.openbrewerydb.org/v1/breweries"
prefix = "/home/jovyan/work/layers/bronze"
file_name = "bronze_breweries.json"
output_path = os.path.join(prefix, file_name)

In [10]:
output_path

'/home/jovyan/work/layers/bronze/bronze_breweries.json'

In [11]:
retry_strategy = Retry(
    total=2,
    backoff_factor=1,
    status_forcelist=[400, 401, 402, 403, 429, 500, 502, 503, 504],
)
adapter = HTTPAdapter(max_retries=retry_strategy)

session = requests.Session()
session.mount("https://", adapter)
session.mount("http://", adapter)

try:
    response = session.get(input_url)
    response.raise_for_status()
    if response.status_code == 200:
        dados_json = response.json()
    else:
        error_message = f"Erro ao acessar os dados: {dados_json}"
        raise Exception(error_message)
except Exception as e:
    error_message = f"Erro ao processar o JSON: {e}"
    raise Exception(error_message)

conteudo_json = json.dumps(dados_json, indent=4, ensure_ascii=False)

try:
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(conteudo_json)
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")

In [19]:
#df_pandas = pd.json_normalize(dados_json)

In [ ]:
#df_pandas.head()